In [1]:
### DRPreter Graph Embedding + PaDEL Descriptor + ResNet

import os
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from pandas import DataFrame
from datetime import datetime
import keras
from keras import Sequential,Input,Model
from keras.layers import Dense, Dropout, Flatten, Reshape, Activation
from keras.layers import Conv1D, MaxPooling1D ,AveragePooling1D
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from keras.layers import LeakyReLU
from keras import backend as K

In [2]:
import pandas as pd
train_data = pd.read_csv('/data/yingfei/cancer_data/train_data.csv')
test_data = pd.read_csv('/data/yingfei/cancer_data/test_data.csv')

In [3]:
from sklearn.model_selection import train_test_split
with open('./Data/train_celllines_v1_1111.txt') as f:
    train_index = list(map(lambda x: x[:-1],f.readlines()))
train_set_index, val_set_index = train_test_split(train_index, test_size=0.1, random_state=42)

In [4]:
train_data = train_data.loc[train_data['auc'] >= 0]
train_data = train_data.reset_index(drop = True)
train_data

,ARXSPAN_ID,DRUG_NAME,ABL1,ACVR1B,AKT1,AKT2,AKT3,ALK,ALOX12B,FAM123B,...,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880,auc
0,ACH-000001,JW-7-24-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.778432
1,ACH-000001,KIN001-260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.951321
2,ACH-000001,NSC-87877,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.840287
3,ACH-000001,PLX-4720,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.936410
4,ACH-000001,ERK5-IN-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.891908
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203913,ACH-001716,KIN001-236,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.956865
203914,ACH-001716,LUMINESPIB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.975168
203915,ACH-001716,NUTLIN-3A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.871995
203916,ACH-001716,SGC0946,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.975417


In [5]:
train_data_new = train_data.loc[train_data.ARXSPAN_ID.isin(train_set_index)].copy()
train_data_new = train_data_new.reset_index(drop = True)
val_data_new = train_data.loc[train_data.ARXSPAN_ID.isin(val_set_index)].copy() 
val_data_new = val_data_new.reset_index(drop = True)

In [6]:
test_data_new = test_data.loc[test_data['auc'] >= 0]
test_data_new = test_data_new.reset_index(drop = True)
test_data_new

,ARXSPAN_ID,DRUG_NAME,ABL1,ACVR1B,AKT1,AKT2,AKT3,ALK,ALOX12B,FAM123B,...,PubchemFP872,PubchemFP873,PubchemFP874,PubchemFP875,PubchemFP876,PubchemFP877,PubchemFP878,PubchemFP879,PubchemFP880,auc
0,ACH-000802,JW-7-24-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.528562
1,ACH-000802,KIN001-260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.930958
2,ACH-000802,NSC-87877,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.759249
3,ACH-000802,PLX-4720,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.936510
4,ACH-000802,ERK5-IN-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.823453
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22873,ACH-000438,KIN001-266,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.975578
22874,ACH-000438,LUMINESPIB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.980529
22875,ACH-000438,NUTLIN-3A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.960501
22876,ACH-000438,SGC0946,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0,0,0,0,0,0,0,0,0,0.970524


In [7]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_padel_features = train_data_new[train_data_new.columns[-2326:-1]]
train_padel_features = scaler.fit_transform(train_padel_features)
train_padel_features = pd.DataFrame(train_padel_features)

In [8]:
val_padel_features = val_data_new[val_data_new.columns[-2326:-1]]
val_padel_features = scaler.transform(val_padel_features)
val_padel_features = pd.DataFrame(val_padel_features)

In [9]:
test_padel_features = test_data_new[test_data_new.columns[-2326:-1]]
test_padel_features = scaler.transform(test_padel_features)
test_padel_features = pd.DataFrame(test_padel_features)

In [10]:
import io
import torch
import pickle

class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == 'torch.storage' and name == '_load_from_bytes':
            return lambda b: torch.load(io.BytesIO(b), map_location='cpu')
        else:
            return super().find_class(module, name)

/home/yingfei/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
with open('embedding/cell_line_embedding_dict_seg_cnv.pickle', 'rb') as file:
    cell_line_embedding_dict = CPU_Unpickler(file).load()

In [12]:
with open('embedding/drug_embedding_dict_seg_cnv.pickle', 'rb') as file:
    drug_embedding_dict = CPU_Unpickler(file).load()

In [14]:
cell_line_embedding_dict_new = {key:value[:, :-1, :].sum(dim=1) for key,value in cell_line_embedding_dict.items()}

In [15]:
train_padel_features.shape

(183501, 2325)

In [16]:
train_padel_features = train_padel_features.to_numpy().reshape(train_padel_features.shape[0], train_padel_features.shape[1], 1)
val_padel_features = val_padel_features.to_numpy().reshape(val_padel_features.shape[0], val_padel_features.shape[1], 1)
test_padel_features = test_padel_features.to_numpy().reshape(test_padel_features.shape[0], test_padel_features.shape[1], 1)

In [17]:
print(train_padel_features.shape, val_padel_features.shape, test_padel_features.shape)

(183501, 2325, 1) (20417, 2325, 1) (22878, 2325, 1)


In [18]:
train_cell_emb = []
for i in range(len(train_data_new)):
    cell_line_id = train_data_new['ARXSPAN_ID'][i]
    train_cell_emb.append(cell_line_embedding_dict_new[cell_line_id].detach().numpy()[0])

In [19]:
val_cell_emb = []
for i in range(len(val_data_new)):
    cell_line_id = val_data_new['ARXSPAN_ID'][i]
    val_cell_emb.append(cell_line_embedding_dict_new[cell_line_id].detach().numpy()[0])

In [20]:
test_cell_emb = []
for i in range(len(test_data_new)):
    cell_line_id = test_data_new['ARXSPAN_ID'][i]
    test_cell_emb.append(cell_line_embedding_dict_new[cell_line_id].detach().numpy()[0])

In [21]:
train_cell_emb = np.array(train_cell_emb)
train_cell_emb = train_cell_emb.reshape(train_cell_emb.shape[0], train_cell_emb.shape[1], 1)
val_cell_emb = np.array(val_cell_emb)
val_cell_emb = val_cell_emb.reshape(val_cell_emb.shape[0], val_cell_emb.shape[1], 1)
test_cell_emb = np.array(test_cell_emb)
test_cell_emb = test_cell_emb.reshape(test_cell_emb.shape[0], test_cell_emb.shape[1], 1)

In [22]:
print(train_cell_emb.shape, val_cell_emb.shape, test_cell_emb.shape)

(183501, 256, 1) (20417, 256, 1) (22878, 256, 1)


In [23]:
train_label = train_data_new['auc']
print(f'Minimum train y value: {min(train_label)}, \
      Maximum train y value: {max(train_label)}')

val_label = val_data_new['auc']
print(f'Minimum validation y value: {min(val_label)}, \
      Maximum validation y value: {max(val_label)}')

test_label = test_data_new['auc']
print(f'Minimum test y value: {min(test_label)}, \
      Maximum test y value: {max(test_label)}')

Minimum train y value: 0.004496,       Maximum train y value: 0.999883
Minimum validation y value: 0.010477,       Maximum validation y value: 0.997684
Minimum test y value: 0.013524,       Maximum test y value: 0.998284


In [24]:
### Train test split
# from sklearn.model_selection import train_test_split
# train_padel_features
# train_padel_features, val_padel_features, train_y, val_y = train_test_split(
#     train_padel_features, train_label, random_state = 42)

In [25]:
# train_cell_emb, val_cell_emb, train_y, val_y = train_test_split(
#     train_cell_emb, train_label, random_state = 42)

In [27]:
train_padel_features = np.nan_to_num(train_padel_features)
train_cell_emb = np.nan_to_num(train_cell_emb)
train_y = np.nan_to_num(train_label)

In [28]:
val_padel_features = np.nan_to_num(val_padel_features)
val_cell_emb = np.nan_to_num(val_cell_emb)
val_y = np.nan_to_num(val_label)

In [29]:
test_padel_features = np.nan_to_num(test_padel_features)
test_cell_emb = np.nan_to_num(test_cell_emb)
test_y = np.nan_to_num(test_label)

In [30]:
### Hyperparameters set
params_lst = ["learning_rate", "batch_size", "optimizer"]
params_value_dict = {"learning_rate": [5e-5, 1e-4, 2e-4], 
                     "batch_size": [64, 128, 256], 
                     "optimizer": ['sgd','adam']}
import itertools as it

allparams = params_value_dict
combinations = it.product(*(params_value_dict[param] for param in allparams))
combinations_lst = list(combinations)
print(len(combinations_lst))

hyper_param_dict = {}
for i in range(len(combinations_lst)):
    hyper_param_dict[i] = {}
    for j in range(len(params_lst)):
        hyper_param_dict[i][params_lst[j]] = combinations_lst[i][j]

18


In [34]:
import json
from keras.models import model_from_json, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from keras.layers import concatenate
workdir = "../resnet_ht_models/emb_model_graph_4"

model_eval_dict = {}
model_dict = {}
training_epochs = 20
model_ht_history = {}

for i in hyper_param_dict:
    if i <= 13: ### run the remaining models
        continue
    params_dict = hyper_param_dict[i]
    learning_rate=params_dict["learning_rate"] 
    batch_size=params_dict["batch_size"] 
    opt_name=params_dict["optimizer"]
    
    # hyper parameters
    num_classes = 1
    if opt_name == 'adam':
        optimizer = keras.optimizers.Adam(learning_rate)
    else: # sgd
        optimizer = keras.optimizers.SGD(learning_rate)
    
    with K.tf.device('/GPU:0'): # model compile
        # inputs = Input(shape=(train_X.shape[1],1),name='inputs')
        
        cell_input = Input(shape=(256,1),  name='cell_input') 
        drug_input = Input(shape=(2325,1), name='drug_input')
        inputs = concatenate([cell_input, drug_input],axis=1) 

        x = Conv1D(16, kernel_size=3, strides=2, padding="same")(inputs)
        x = MaxPooling1D(pool_size=5)(x)
        x = BatchNormalization()(x)
    #     y = x
        x = Activation('tanh')(x)

        x = Conv1D(16, kernel_size=3, strides=1, padding="same")(x)
    #     x = MaxPooling1D(pool_size=5)(x)
        x = BatchNormalization()(x)
        y = x
        x = Activation('relu')(x)

        x = Conv1D(16, kernel_size=3, strides=1, padding="same")(x)
    #     x = MaxPooling1D(pool_size=5)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv1D(16, kernel_size=3, strides=1, padding="same")(x)
    #     x = MaxPooling1D(pool_size=5)(x)
        x = BatchNormalization()(x)
        x = keras.layers.add([x,y])
        x = Activation('relu')(x)


        x = Conv1D(16, kernel_size=3, strides=1, padding="same")(x)
    #     x = MaxPooling1D(pool_size=5)(x)
        x = BatchNormalization()(x)
        y = x
        x = Activation('relu')(x)


        x = Conv1D(16, kernel_size=3, strides=1, padding="same")(x)
    #     x = MaxPooling1D(pool_size=5)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv1D(16, kernel_size=3, strides=1, padding="same")(x)
    #     x = MaxPooling1D(pool_size=5)(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)


        x = Conv1D(16, kernel_size=3, strides=1, padding="same")(x)
    #     x = MaxPooling1D(pool_size=5)(x)
        x = BatchNormalization()(x)
        x = keras.layers.add([x,y])
        x = Activation('relu')(x)

        x = Conv1D(32, kernel_size=3, strides=2, padding="same")(x)
    #     x = MaxPooling1D(pool_size=5)(x)
        x = BatchNormalization()(x)
        y = x
        x = Activation('relu')(x)

        x = Conv1D(32, kernel_size=3, strides=1, padding="same")(x)
    #     x = MaxPooling1D(pool_size=5)(x)
    #     x = BatchNormalization()(x)

        x = Conv1D(32, kernel_size=3, strides=1, padding="same")(x)
    #     x = MaxPooling1D(pool_size=5)(x)
        x = BatchNormalization()(x)
        x = keras.layers.add([x,y])
        x = Activation('relu')(x)


        x = Conv1D(32, kernel_size=3, strides=1, padding="same")(x)
    #     x = MaxPooling1D(pool_size=5)(x)
        x = BatchNormalization()(x)
        y = x
        x = Activation('relu')(x)

        x = Conv1D(32, kernel_size=3, strides=1, padding="same")(x)
    #     x = MaxPooling1D(pool_size=5)(x)
        x = BatchNormalization()(x)
        x = keras.layers.add([x,y])
        x = Activation('relu')(x)

        x = Conv1D(32, kernel_size=3, strides=1, padding="same")(x)
    #     x = MaxPooling1D(pool_size=5)(x)
        x = BatchNormalization()(x)
        y = x
        x = Activation('relu')(x)

        x = Conv1D(32, kernel_size=3, strides=1, padding="same")(x)
    #     x = MaxPooling1D(pool_size=5)(x)
        x = BatchNormalization()(x)
        x = keras.layers.add([x,y])
        x = Activation('relu')(x)

        x = Conv1D(64, kernel_size=3, strides=2, padding="same")(x)
    #     x = MaxPooling1D(pool_size=5)(x)
        x = BatchNormalization()(x)
        y = x
        x = Activation('relu')(x)

        x = Conv1D(64, kernel_size=3, strides=1, padding="same")(x)
    #     x = MaxPooling1D(pool_size=5)(x)
    #     x = BatchNormalization()(x)

        x = Conv1D(64, kernel_size=3, strides=1, padding="same")(x)
    #     x = MaxPooling1D(pool_size=5)(x)
        x = BatchNormalization()(x)
        x = keras.layers.add([x,y])
        x = Activation('relu')(x)


        x = Conv1D(64, kernel_size=3, strides=1, padding="same")(x)
    #     x = MaxPooling1D(pool_size=5)(x)
        x = BatchNormalization()(x)
        y = x
        x = Activation('relu')(x)

        x = Conv1D(64, kernel_size=3, strides=1, padding="same")(x)
    #     x = MaxPooling1D(pool_size=5)(x)
        x = BatchNormalization()(x)
        x = keras.layers.add([x,y])
        x = Activation('relu')(x)

        x = Conv1D(64, kernel_size=3, strides=1, padding="same")(x)
    #     x = MaxPooling1D(pool_size=5)(x)
        x = BatchNormalization()(x)
        y = x
        x = Activation('relu')(x)

        x = Conv1D(64, kernel_size=3, strides=1, padding="same")(x)
    #     x = MaxPooling1D(pool_size=5)(x)
        x = BatchNormalization()(x)
        x = keras.layers.add([x,y])
        x = Activation('relu')(x)

    #     x = AveragePooling1D(pool_size=8)(x)
        x = Flatten()(x)
        x = Dense(units=2048, name='dense1'  ) (x)
        x = BatchNormalization()(x)
        x = Dropout(0.1, name='dropout1') (x)
        x = Activation('relu')(x)

    #    x = Reshape((300,1))(x)

    #    x = Conv1D(30, kernel_size=150, strides=1, activation = 'relu')(x)
    #    x = MaxPooling1D(pool_size=2)(x)
    #    x = BatchNormalization()(x)

        x = Dense(units=1024, name='dense5'  ) (x)
        x = BatchNormalization()(x)
        y = x
        x = Dropout(0.1, name='dropout5') (x)
        x = Activation('relu')(x)

        x = Dense(units=512, name='dense6'  ) (x)
        x = BatchNormalization()(x)
        x = Dropout(0.1, name='dropout6') (x)
        x = Activation('relu')(x)

        x = Dense(units=1024, name='dense7'  ) (x)
        x = BatchNormalization()(x)
        x = Dropout(0.1, name='dropout7') (x)
        x = keras.layers.add([x,y])
        x = Activation('relu')(x)

        x = Dense(units=512, name='dense8'  ) (x)
        x = BatchNormalization()(x)
        x = Dropout(0.1, name='dropout8') (x)
        x = Activation('relu')(x)

        x = Dense(units=256, name='dense9'  ) (x)
        x = BatchNormalization()(x)
        x = Dropout(0.1, name='dropout9') (x)
        x = Activation('relu')(x)

        x = Dense(units=128, name='dense10'  ) (x)
        x = BatchNormalization()(x)
        y = x
        x = Dropout(0.1, name='dropout10') (x)
        x = Activation('relu')(x)


        predictions = Dense(1, activation='linear', name='predictions', kernel_initializer='he_normal')(x)

        # model = Model(inputs=inputs, outputs=predictions, name='Test_v2_DNN')
        model = Model(inputs=[cell_input , drug_input], outputs=[predictions])
        model.compile(loss=keras.losses.mean_squared_error,
                      optimizer=optimizer,
                      metrics=['mse','mae'])
        
        # model training
        StartTime8 = datetime.now()
        print("StartTime :", StartTime8)
        with K.tf.device('/GPU:0'):
            # model_train = model.fit(train_X, train_y, batch_size=batch_size,epochs=training_epochs,verbose=1,
            #                     validation_data=(val_X, val_y))
            model_train = model.fit({'cell_input': train_cell_emb, 'drug_input': train_padel_features}, train_y, 
                                    batch_size=batch_size,epochs=training_epochs,verbose=1,
                                    validation_data=({'cell_input': val_cell_emb, 
                                                      'drug_input': val_padel_features}
                                                     , val_y))

        EndTime8 = datetime.now()
        print("EndTime :", EndTime8)
    model.save_weights(workdir+ f'/model_{i}_new.h5')
    with open(workdir + f'/model_architecture_{i}_new.json', 'w') as f:
        f.write(model.to_json())
        
    # evaluation
    print(f"Evaluating model {i}...")
    test_score = model.evaluate({'cell_input': test_cell_emb, 'drug_input': test_padel_features}, test_y, verbose=1)
    model_ht_history[(learning_rate, batch_size)] = model
    loss, mse, mae = test_score
    print("loss=%.6f, mse=%.6f, mae=%.6f"%(loss, mse, mae))
    
    model_dict[i] = model
    model_eval_dict[i] = {"loss":loss, "mse":mse, "mae":mae}

StartTime : 2023-03-28 18:48:04.491207
Epoch 1/20
1434/1434 [==============================] - 331s 221ms/step - loss: 0.5727 - mse: 0.5727 - mae: 0.5934 - val_loss: 0.1901 - val_mse: 0.1901 - val_mae: 0.3491
Epoch 2/20
1434/1434 [==============================] - 322s 225ms/step - loss: 0.3017 - mse: 0.3017 - mae: 0.4340 - val_loss: 0.1227 - val_mse: 0.1227 - val_mae: 0.2818
Epoch 3/20
1434/1434 [==============================] - 322s 224ms/step - loss: 0.2127 - mse: 0.2127 - mae: 0.3631 - val_loss: 0.0997 - val_mse: 0.0997 - val_mae: 0.2542
Epoch 4/20
1434/1434 [==============================] - 322s 225ms/step - loss: 0.1560 - mse: 0.1560 - mae: 0.3102 - val_loss: 0.0682 - val_mse: 0.0682 - val_mae: 0.2118
Epoch 5/20
1434/1434 [==============================] - 326s 227ms/step - loss: 0.1224 - mse: 0.1224 - mae: 0.2737 - val_loss: 0.0599 - val_mse: 0.0599 - val_mae: 0.1997
Epoch 6/20
1434/1434 [==============================] - 350s 244ms/step - loss: 0.1025 - mse: 0.1025 - mae: 0.2

717/717 [==============================] - 293s 409ms/step - loss: 0.2200 - mse: 0.2200 - mae: 0.3704 - val_loss: 0.1139 - val_mse: 0.1139 - val_mae: 0.2755
Epoch 7/20
717/717 [==============================] - 292s 407ms/step - loss: 0.1912 - mse: 0.1912 - mae: 0.3451 - val_loss: 0.0974 - val_mse: 0.0974 - val_mae: 0.2527
Epoch 8/20
717/717 [==============================] - 295s 411ms/step - loss: 0.1698 - mse: 0.1698 - mae: 0.3245 - val_loss: 0.0840 - val_mse: 0.0840 - val_mae: 0.2333
Epoch 9/20
717/717 [==============================] - 297s 414ms/step - loss: 0.1504 - mse: 0.1504 - mae: 0.3046 - val_loss: 0.0811 - val_mse: 0.0811 - val_mae: 0.2307
Epoch 10/20
717/717 [==============================] - 297s 414ms/step - loss: 0.1357 - mse: 0.1357 - mae: 0.2894 - val_loss: 0.0795 - val_mse: 0.0795 - val_mae: 0.2317
Epoch 11/20
717/717 [==============================] - 299s 417ms/step - loss: 0.1241 - mse: 0.1241 - mae: 0.2769 - val_loss: 0.0727 - val_mse: 0.0727 - val_mae: 0.2201
E

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

